In [2]:
# our nice list of imports is in here
#i hate this, but the notebook won't recognize our custom modules w/o it
import sys
sys.path.append("..")
import lib
from lib import utils

import os
import cv2
import urllib
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow.keras.models import load_model


from mtcnn.mtcnn import MTCNN

Using TensorFlow backend.


# Load A Model

In [ ]:
def extr_pmask(pred, face):
    """
        @param pred is a prediction from our project model
        @param face is a face object returned from the MTCNN detector
        @return results after non-standard regularization
    """
    p_mask = pred[0][1]
    p_nomask = pred[0][0]
    p_face = f['confidence']
    p_noface = 1 - p_face
    
    results = [
        (p_face * p_nomask) / p_face,
        (p_noface * p_mask) / p_face
    ]
    
    
    return results

In [79]:
kaggle_model = load_model('../models/kaggle_relu_softmax_1e31e5')
presentation_model = load_model('../models/mnm/presentation/presentation.checkpoint')

# Using The Model With Video Feed

+ We have two available facial detection algorithms: 
  1. Haar Cascade - faster detection, but only tuned for full, frontal views of faces.
  2. MTCNN - slower detection, but returns extra features and can detect faces from a variety of angles -- not only the front.


In [84]:
# use haar cascade to detect faces (frontal)
# HC_FRONTAL_FACE = os.path.join(HC_DATA_DIR, 
#                                'haarcascade_frontalface_default.xml')
# classifier = cv2.CascadeClassifier(HC_FRONTAL_FACE)


# use MTCNN to detect faces
classifier = MTCNN()


labels_dict={ 0:'No Mask', 1:'Mask' }
color_dict={ 0: (0,0,255), 1: (0,255,0) }

size = 8

# open webcam feed
# cap = cv2.VideoCapture(0)

# alternatively, open feed from a file
cap = cv2.VideoCapture('../memask4.mov')

# EVEN from a mobile device (UNTESTED)
# cap = cv.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if frame is not None:
        # frame = cv2.flip(frame, 1, 1) # flip so view looks more natural live (?)
        

        # detection is a little faster on smaller images -- so let's make it small
        small = cv2.resize(frame, (frame.shape[1] // size, frame.shape[0] // size))

        # detect faces with haar cascades
        # faces = classifier.detectMultiScale(small)
        
        # detect faces with MTCNN
        faces = classifier.detect_faces(small)

        for f in faces:
            # reverse the scale-down of the frame for bounding box (haar)
#             (x, y, w, h) = [v * size for vstack in f] 
#             keypoints = None
            
            # reverse the scale-down of the frame for bounding box (MTCNN)
            (x, y, w, h) = [v * size for v in f['box']]
            keypoints = f['keypoints']
            p_face = f['confidence']
            p_noface = 1 - p_face
                
        
            face = frame[y:y+h, x:x+w]
            resized = cv2.resize(face, (50,50))
            normalized = resized/255.0
            
            reshaped = np.reshape(normalized,(-1,50,50,1))
            reshaped = np.vstack([reshaped])
            result1 = kaggle_model.predict(reshaped)
            
            reshaped2 = np.reshape(normalized, (-1, 50, 50, 3))
            reshaped2 = np.vstack([reshaped2])
            result2 = presentation_model.predict(reshaped2)
            result2[0][0] = (p_face * result2[0][0]) / p_face
            result2[0][1] = (p_noface * result2[0][1]) / p_face
            
            # print(result1)
            # print(result2)
            
            
            left = frame
            right = np.copy(frame)
            frames = [left, right]

            label1 = np.argmax(result1, axis=1)[0]
            label2 = np.argmax(result2, axis=1)[0]
            # print("Kaggle Prediction: {}\nPresentation Prediction: {}".format(labels_dict[label1], labels_dict[label2]))

            cv2.rectangle(left, (x,y), (x+w,y+h), color_dict[label1], 2)
            cv2.rectangle(left, (x,y-40), (x+w,y), color_dict[label1], -1)
            cv2.putText(
                left, 
                labels_dict[label1], 
                (x+5, y-10), 
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (255,255,255),
                2
            )
            cv2.rectangle(left, (x, y+h), (x+w,y+h+40), color_dict[label1], -1)
            cv2.putText(
                left,
                "Top Ranked Kaggle Model",
                (x+5, y+h+20),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                2
            )
            
            cv2.rectangle(right, (x,y), (x+w,y+h), color_dict[label2], 2)
            cv2.rectangle(right, (x,y-40), (x+w,y), color_dict[label2], -1)
            cv2.putText(
                right, 
                labels_dict[label2], 
                (x+5, y-10), 
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (255,255,255),
                2
            )
            cv2.rectangle(right, (x, y+h), (x+w,y+h+40), color_dict[label2], -1)
            cv2.putText(
                right,
                "Our Model",
                (x+65, y+h+20),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                2
            )
            
            # extra fun when using MTCNN :)
            for frame in frames:
                if keypoints is not None:
                    cv2.circle(
                        frame, 
                        (tuple([k * size for k in keypoints['left_eye']])), 
                        2, (255,0,255), 2
                    )
                    cv2.circle(
                        frame, 
                        (tuple([k * size for k in keypoints['right_eye']])), 
                        2, (255,0,255), 2
                    )
                    cv2.circle(
                        frame, 
                        (tuple([k * size for k in keypoints['nose']])), 
                        2, (255,0,255), 2
                    )
                    cv2.circle(
                        frame, 
                        (tuple([k * size for k in keypoints['mouth_left']])), 
                        2, (255,0,255), 2
                    )
                    cv2.circle(
                        frame, 
                        (tuple([k * size for k in keypoints['mouth_right']])), 
                        2, (255,0,255), 2)
            # end of extra fun with MTCNN

            
        view = np.hstack(frames)
        cv2.imshow('LIVE DETECTION ACTIVE', view)
        key = cv2.waitKey(10)

        if key == 27: # if Esc key pressed -- end feed
            break
        
#         clear_output(wait=True)
    else:
        break

In [27]:
# stop video
cap.release()

# close windows
cv2.destroyAllWindows()